<a href="https://colab.research.google.com/github/Vikram12301/Time-Series-Classification-for-FordA/blob/main/Rocket_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def create_kernel(input_length,num_kernals):
  kernel_length_choices = np.array((7, 9, 11), dtype = np.int32)
  kernel_lengths = np.random.choice(kernel_length_choices,num_kernels)
  weights = np.zeros(np.sum(kernel_lengths),dtype=np.float64)
  biases = np.zeros(num_kernels,dtype=np.float64)
  dilations = np.zeros(num_kernels,dtype=np.int32)
  paddings = np.zeros(num_kernels,dtype=np.int32)
  start = 0
  for i in range(num_kernals):
    length = kernel_lengths[i]
    w = np.random.normal(0,1,length)
    end = start + length
    weights[start:end] = w - np.mean(w)
    biases[i] = np.random.uniform(-1, 1)
    dilation = 2 ** np.random.uniform(0,np.log2((input_length-1)/(length-1)))
    dilation = np.int32(dilation)
    dilations[i] = dilation
    padding = ((length - 1)*dilation)//2 if np.random.randint(2)== 1 else 0
    paddings[i] = padding
    start = end
    return weights, lengths, biases, dilations, paddings

In [3]:
def kernalise(X, weights, length, bias, dilation, padding):
  input_length = len(X)
  output_length = (input_length+(2*padding))-((length-1)*dilation)
  ppv = 0                                                               # To count the positive values of sum
  max = np.NINF                                                         # To find the maximum sum 
  end = (input_length+padding)-((length-1)*dilation)
  for i in range(-padding, end):
    sum = bias
    index = i
    for j in range(length):
      if index > -1 and index < input_length:
        sum = sum + weights[j] * X[index]
      index = index + dilation
    if sum > max:
      max = sum
    if sum > 0:
      ppv += 1
    return ppv/output_length,max

In [4]:
def apply_kernels(X, kernels):
  weights, lengths, biases, dilations, paddings = kernels
  num_examples, _ = X.shape
  num_kernels = len(lengths)
  _X = np.zeros((num_examples, num_kernels * 2), dtype = np.float64)
  for i in range(num_examples):
    a1 = 0 # for weights
    a2 = 0 # for features
    for j in range(num_kernels):
      b1 = a1 + lengths[j]
      b2 = a2 + 2
      _X[i, a2:b2] = kernelise(X[i], weights[a1:b1], lengths[j], biases[j], dilations[j], paddings[j])
      a1 = b1
      a2 = b2
  return _X